# Autoscaling Seldon Deployments


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
    - The cluster should have `heapster` and `metric-server` running in the `kube-system` namespace
    - For Minikube run:
    ```
    minikube addons enable metrics-server
    minikube addons enable heapster
    ```
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [12]:
!kubectl create namespace seldon

namespace/seldon created


In [13]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [14]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [15]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [16]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [30]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Tue Mar 19 17:45:01 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s


NOTES:
NOTES: TODO




In [32]:
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Tue Mar 19 17:54:27 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        0s

==> v1/Role
NAME          AGE
ambassador    0s
seldon-local  0s

==> v1/RoleBinding
NAME        AGE
seldon      0s
ambassador  0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.101.226.47   <none>       80:31875/TCP,443:32368/TCP     0s
seldon-core-ambassador-admin  NodePort   10.109.3.143    <none>       8877:31796/TCP                 0s
seldon-core-seldon-apiserver  NodePort   10.111.58.187   <none>       8080:32545/TCP,5000:32649/TCP  0s
seldon-core-redis             ClusterIP  10.109.127.109  <none>       6379/TCP                       0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1  

In [33]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Create model with autoscaler

To create a model with an HorizontalPodAutoscaler there are three steps:


  1. Ensure you have a resource request for the metric you want to scale on if it is a standard metric such as cpu or memory, e.g.:
     ```
      "resources": {
         "requests": {
            "cpu": "0.5"
         }
      }
     ```
  1. Add an HPA Spec refering to this Deployment, e.g.:
     ```
     "hpaSpec":
		    {
			"minReplicas": 1,
			"maxReplicas": 4,
			"metrics": 
			    [ {
				"type": "Resource",
				"resource": {
				    "name": "cpu",
				    "targetAverageUtilization": 10
				}
			    }]
		    },
     ```

The full SeldonDeployment spec is shown below.

In [20]:
!pygmentize model_with_hpa.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "cpu": "0.5"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSeconds": 1
                    },
	

In [34]:
!kubectl create -f model_with_hpa.json

seldondeployment.machinelearning.seldon.io/seldon-model created


## Create Load

In [35]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust

node/minikube labeled


In [36]:
!helm install ../../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://test-deployment-seldon-model:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=1 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=1

NAME:   loadtest
LAST DEPLOYED: Tue Mar 19 17:56:59 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-vbklw   0/1    ContainerCreating  0         0s
locust-master-1-bxbl2  0/1    ContainerCreating  0         0s

==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP    EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.108.0.161  <none>       5557:32546/TCP,5558:31904/TCP,8089:32010/TCP  0s




After a few mins you should see the deployment `my-dep` scaled to 4 deployments

In [37]:
!kubectl get pods,deployments,hpa

NAME                                                      READY   STATUS    RESTARTS   AGE
pod/locust-master-1-bxbl2                                 1/1     Running   0          11m
pod/locust-slave-1-vbklw                                  1/1     Running   0          11m
pod/seldon-core-ambassador-847f7d4c4f-8v5p2               1/1     Running   0          14m
pod/seldon-core-redis-7d64dc686b-sth8q                    1/1     Running   0          14m
pod/seldon-core-seldon-apiserver-7c9898d988-fvx9s         1/1     Running   0          14m
pod/seldon-core-seldon-cluster-manager-68ff4ccfcf-2hnct   1/1     Running   0          14m
pod/test-deployment-example-7cd068f-59c8b498db-5d422      2/2     Running   0          39s
pod/test-deployment-example-7cd068f-59c8b498db-kkdhj      2/2     Running   0          38s
pod/test-deployment-example-7cd068f-59c8b498db-qrcsz      2/2     Running   0          38s
pod/test-deployment-example-7cd068f-59c8b498db-tzzsq      2/2     Running   0          13m

## Remove Load
After 5-10 mins you should see the deployments replicas decrease to 1

In [38]:
!helm delete loadtest --purge

release "loadtest" deleted


In [39]:
!kubectl get pods,deployments,hpa

NAME                                                      READY   STATUS    RESTARTS   AGE
pod/seldon-core-ambassador-847f7d4c4f-8v5p2               1/1     Running   0          26m
pod/seldon-core-redis-7d64dc686b-sth8q                    1/1     Running   0          26m
pod/seldon-core-seldon-apiserver-7c9898d988-fvx9s         1/1     Running   0          26m
pod/seldon-core-seldon-cluster-manager-68ff4ccfcf-2hnct   1/1     Running   0          26m
pod/test-deployment-example-7cd068f-59c8b498db-tzzsq      2/2     Running   0          25m

NAME                                                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/seldon-core-ambassador               1/1     1            1           26m
deployment.extensions/seldon-core-redis                    1/1     1            1           26m
deployment.extensions/seldon-core-seldon-apiserver         1/1     1            1           26m
deployment.extensions/seldon-core-seldon-cluster-manager   1/1     1 